In [1]:
import os
import numpy as np
import xarray as xr
import cftime
import pandas as pd
import glob
import matplotlib.pyplot as plt

import xesmf as xe

import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import fates_calibration_library.analysis_functions as analysis
import fates_calibration_library.ilamb_functions as ilamb

In [2]:
# Setup PBSCluster
cluster = PBSCluster(
    cores=1,                                                   # The number of cores you want
    memory='25GB',                                             # Amount of memory
    processes=1,                                               # How many processes
    queue='casper',                                            # The type of queue to utilize
    local_directory='/glade/work/afoster',                     # Use your local directory
    resource_spec='select=1:ncpus=1:mem=25GB',                 # Specify resources
    log_directory='/glade/derecho/scratch/afoster/dask_logs',  # log directory
    account='P08010000',                                       # Input your project ID here
    walltime='02:00:00',                                       # Amount of wall time
    interface='ext')

In [3]:
cluster.scale(30)
dask.config.set({
    'distributed.dashboard.link': 'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
})
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.100:35525,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [59]:
fates_vars = ['FATES_FRACTION', 'FATES_GPP', 'FATES_LAI', 'QVEGE',
              'QVEGT', 'EFLX_LH_TOT', 'FSH', 'FSR', 'FSDS', 'FSA',
              'FIRE', 'FLDS', 'FCTR', 'FCEV', 'FGEV', 'BTRANMN', 
              'FGR', 'SOILWATER_10CM', 'TWS', 'QRUNOFF', 'SNOWDP',
              'TV', 'TG', 'TSA']

clm_vars = ['FPSN', 'TLAI', 'QVEGE',
              'QVEGT', 'EFLX_LH_TOT', 'FSH', 'FSR', 'FSDS', 'FSA',
              'FIRE', 'FLDS', 'FCTR', 'FCEV', 'FGEV', 'BTRANMN', 
              'FGR', 'SOILWATER_10CM', 'TWS', 'QRUNOFF', 'SNOWDP',
              'TV', 'TG', 'TSA']

out_vars = ['GPP', 'LAI', 'EFLX_LH_TOT', 'FSH', 'EF', 'SOILWATER_10CM', 'ASA',
        'FSR', 'FSA', 'FIRE', 'RLNS', 'RN', 'BTRANMN', 'TV']

In [60]:
# sparse grid clustering file
grid_dir = '/glade/work/afoster/FATES_calibration/surfdata'
grid_file = 'clusters.clm51_PPEn02ctsm51d021_2deg_GSWP3V1_leafbiomassesai_PPE3_hist.annual+sd.400.nc'
sparse_grid = xr.open_dataset(os.path.join(grid_dir, grid_file))

# sparse grid whittaker biomes
whittaker_ds = xr.open_dataset('/glade/work/afoster/FATES_calibration/observations/whittaker/whitkey.nc')

# fetch the sparsegrid landarea - needed for unit conversion
land_area_file = '/glade/work/afoster/FATES_calibration/CLM5PPE/postp/sparsegrid_landarea.nc'
land_area = xr.open_dataset(land_area_file).landarea  #km2

In [61]:
hist_dir = '/glade/derecho/scratch/afoster/archive/ctsm60SP_bigleaf_sparsegrid/lnd/hist'
files = sorted(glob.glob(f"{hist_dir}/*clm2.h0*.nc"))

ds = analysis.get_clm_ds(files, clm_vars, 2000, fates=False)
ds['biome'] = whittaker_ds.biome
ds['biome_name'] = whittaker_ds.biome_name

In [62]:
import functools

ds = xr.open_mfdataset(
    files,
    combine="nested",
    concat_dim="time",
    preprocess=functools.partial(analysis.preprocess, data_vars=clm_vars),
    parallel=True,
    autoclose=True,
)

In [40]:
file = xr.open_dataset('/glade/work/afoster/old_calib/hist_500/ctsm51FATES_SP_OAAT_SatPhen_derecho_2000_FATES_LH_500.nc')

In [53]:
file2 = xr.open_dataset('/glade/derecho/scratch/afoster/archive/ctsm60SP_bigleaf_fullgrid/lnd/hist/ctsm60SP_bigleaf_fullgrid.clm2.h0.0076-02-01-00000.nc')

In [25]:
#plt.scatter(ds.grid1d_lon, ds.grid1d_lat, c=ds.landfrac)

In [ ]:
gpp_map = analysis.global_from_sparse(sparse_grid, ds['GPP'], ds)
gpp_mean = analysis.calculate_annual_mean(gpp_map, 24*60*60).isel(year=19)

lai_map = analysis.global_from_sparse(sparse_grid, ds['LAI'], ds)
lai_mean = analysis.calculate_annual_mean(lai_map, 1/365).isel(year=19)